# Tutorial 5: Assistant Prefill in RadPrompter

In this tutorial, we'll explore how to use assistant prefills in RadPrompter. Assistant prefills are pre-filled responses that guide the model to generate outputs in a specific format.

**Note:** Assistant prefills are not currently supported with the `OpenAIClient`. However, they are supported by other clients such as `vLLMClient` and `OllamaClient`.

## Prompt

As always, we start by importing the `Prompt` class and creating a prompt object from a TOML file:

In [1]:
from radprompter import Prompt

prompt = Prompt('05_Assistant-Prefill.toml')
prompt

The key feature of this TOML file is the `assistant_response_template`:

```toml
assistant_response_template = """
<json>
{
  "{{variable_name}}" : \""
```

This prefill requires the model to generate the output in JSON format. We also included the `{{variable_name}}` to replace this placeholder with each schema's `variable_name` attribute.

We also add a `stop_tags` field in the `[CONSTRUCTOR]` section:

```toml
[CONSTRUCTOR]
system = "rdp(system_prompt)"
user = "rdp(user_prompt_intro + user_prompt_cot)"
response_templates = "rdp(assistant_response_template)"
stop_tags = "</json>"
```

This tells the model to stop generating when it reaches the `</json>` tag, ensuring that it doesn't generate any extra text after the JSON object.

Let's look at the first schema:


In [2]:
prompt.schemas[1]

You can see the prefilled assistant response, preceding model's `[... response ...]`.

## Client and Engine

We'll use the `vLLMClient` and `RadPrompter` engine:

In [3]:
from radprompter import RadPrompter, vLLMClient

client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_5.csv",
    concurrency=1,
    hide_blocks=False,
)

And we run it on our sample reports:

In [4]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

engine(reports)

Processing items: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


The engine will process each report using **ALL** the schemas in the prompt and save the results to `output_tutorial_5.csv`.

In [5]:
import pandas as pd

df = pd.read_csv("output_tutorial_5.csv", index_col='index')
df

,Pulmonary Embolism_response,Laterality_response,Acuity_response,report,file_name
index,,,,,
0,"""Present""\n","<json>\n{\n ""Laterality"" : ""Bilateral""\n}\n","<json>\n{\n ""Acuity"" : ""Acute""\n}\n",Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
1,Present,"<json>\n{\n ""Laterality"" : ""Bilateral""\n}\n","<json>\n{\n ""Acuity"" : ""Not Mentioned""\n}\n",Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
2,"""Present""\n","<json>\n{\n ""Laterality"" : ""Bilateral""\n}\n","<json>\n{\n ""Acuity"" : ""Not Mentioned""\n}\n",Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt
index,Pulmonary Embolism_response,Laterality_response,Acuity_response,report,file_name
0,"""Present""\n","""Bilateral""\n","<json>\n{\n ""Acuity"" : ""Acute""\n",Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
1,"""Present""\n","""Right and Left""\n","<json>\n{\n ""Acuity"" : ""Not Mentioned""\n",Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
2,"""Present""\n","""Bilateral""\n","<json>\n{\n ""Acuity"" : ""Not Mentioned""\n",Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt


As you can see, the model does not always stick to the template and might deviate from that. But, we've got you covered. You can use this simple utility function to sanitize JSON outputs.

In [6]:
sanitized_df = engine.sanitize_json("all")
sanitized_df

,index,Pulmonary Embolism_response,Laterality_response,Acuity_response,report,file_name
0,0,Present,Bilateral,Acute,Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
1,1,Present,Bilateral,Not Mentioned,Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
2,2,Present,Bilateral,Not Mentioned,Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt
3,index,**RECHECK** Pulmonary Embolism_response,**RECHECK** Laterality_response,**RECHECK** Acuity_response,report,file_name
4,0,Present,Bilateral,Acute,Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
5,1,Present,"**RECHECK** ""Right and Left""\n",Not Mentioned,Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
6,2,Present,Bilateral,Not Mentioned,Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt


Finally, we save the log:

In [7]:
engine.save_log("log_tutorial_5.log")

with open("log_tutorial_5.log", "r") as f:
    print(f.read())

RadPrompter Version: 1.0.9
Model: meta-llama/Meta-Llama-3-8B-Instruct
Prompt TOML: /Users/bardiakhosravi/Desktop/GitHub/RadPrompter/tutorials/05_Assistant-Prefill/05_Assistant-Prefill.toml
Prompt Version: 0.1
Prompt Hash: e9d69fb40739d78b3381842b54b77e48
Concurrency Factor: 1
Start Time: 2024-05-19 16:38:01
End Time: 2024-05-19 16:38:04
Duration: 3.0
Number of Items: 3
Average Processing Time: 1.0


-------------------- *** - Prompt Content - *** --------------------
[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter"

[PROMPTS]

system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = """
{{intro_prompt}}
"""

user_prompt_no_cot = """
I want you to extract the following data element from the report: 
{{hint}}

Your final answer elements in a valid JSON format and is between <json> and </json> tags:

<json>
{
  "{{variable_name}}": "answer"
}
</json>
"""

assistant

Assistant prefills are a powerful feature that allow us to guide the model's output into a specific format. By providing a template in the `assistant_templates` field, we can ensure that the model's responses are structured in a way that's easy to parse and process downstream.

In this tutorial, we used assistant prefills to have the model generate its outputs as JSON objects. This can greatly simplify the task of extracting the relevant information from the model's responses.